In [ ]:
# auto-reload all helper files
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Imports
import config
import numpy as np
from tensorflow.keras.models import load_model
from data import get_cifar10_data
from model import build_model
from train import compile_model, train_model, evaluate_model
from metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# load data
x_train, y_train, x_test, y_test = get_cifar10_data()

c:\Users\yogan\Documents\JupyterNotebook_ironhack\.venv\Lib\site-packages\keras\src\datasets\cifar.py:18: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  d = cPickle.load(f, encoding="bytes")


In [ ]:
# Build and compile - # Run ONCE per model lifecycle as it resets model weights

model = build_model()
model = compile_model(model)

In [ ]:
# Train model (saves models/model.keras)

print(">>> TRAINING STARTING <<<")

history = train_model(
    model,
    x_train,
    y_train,
    batch_size=config.BATCH_SIZE,
    epochs=config.EPOCHS,
)

Epoch 1/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.2780 - loss: 2.0145 - val_accuracy: 0.2986 - val_loss: 1.9728
Epoch 2/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - accuracy: 0.3731 - loss: 1.7891 - val_accuracy: 0.3490 - val_loss: 1.8800
Epoch 3/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - accuracy: 0.4228 - loss: 1.6482 - val_accuracy: 0.2362 - val_loss: 2.2311
Epoch 4/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.4635 - loss: 1.5269 - val_accuracy: 0.4208 - val_loss: 1.6166
Epoch 5/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - accuracy: 0.4959 - loss: 1.4282 - val_accuracy: 0.4896 - val_loss: 1.4322
Epoch 6/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 15s 21ms/step - accuracy: 0.5229 - loss: 1.3543 - val_accuracy: 0.4696 - val_loss: 1.4874
Epoch 7/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.5436 - loss: 1.2955 - val_accuracy: 0.4246 - val_loss: 1.6618
Epoch 8/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 15s 21ms/step - accuracy: 0.5622 - loss: 1.2492 - 

In [10]:
# 4. Reload saved model for evaluation

if config.LOAD_SAVED_MODEL:
    model = load_model("models/model.keras")

Test accuracy: 0.6093999743461609


In [ ]:
# Evaluate Testing Data

test_loss, test_accuracy = evaluate_model(model, x_test, y_test)
print(f"Test accuracy: {test_accuracy:.2%}")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
# Model Evaluation Metrics

# Evaluate model on unseen test data
y_pred_prob = model.predict(x_test)

# Selecting predicted class with highest probability per sample
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = y_test

# Visualizing classification performance using a confusion matrix (true vs predicted labels)
cm = confusion_matrix(y_true, y_pred, config.NUM_CLASSES)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=config.CLASS_NAMES, yticklabels=config.CLASS_NAMES)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


In [7]:
# Training history visualization (loss)

val_loss = history.history['val_loss']
loss = history.history['loss']

# Dynamically generate epoch axis to support variable training length (e.g. EarlyStopping)
epoch_axis = range(1, len(history.history["loss"]) + 1)

plt.figure()
plt.plot(epoch_axis, loss, label="Training loss")
plt.plot(epoch_axis, val_loss, label="Validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Training vs Validation Loss")
plt.show()

# Extract accuracy metrics from training history
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

plt.figure()
plt.plot(epoch_axis, accuracy, label="Training accuracy")
plt.plot(epoch_axis, val_accuracy, label="Validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Training vs Validation Accuracy")
plt.show()


# Snapshot final epoch performance
print(f'Training Loss: {loss[-1]:4f}')
print(f'Validation Loss: {val_loss[-1]:4f}')
print(f'Training Accuracy: {accuracy[-1]:.2%}')
print(f'Validation Accuracy: {val_accuracy[-1]:.2%}')
print('\n')

# Print out the best epoch where val_loss was at a minimum
best_epoch = np.argmin(val_loss)

# Diagnostic metrics: best epoch based on minimum validation loss
print(f"Best Epoch: {best_epoch + 1}")
print(f"Best Training Loss: {loss[best_epoch]:.4f}")
print(f"Best Validation Loss: {val_loss[best_epoch]:.4f}")
print(f"Best Training Accuracy: {accuracy[best_epoch]:.2%}")
print(f"Best Validation Accuracy: {val_accuracy[best_epoch]:.2%}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
